In [1]:
!pip install seaborn
!pip install dill

In [2]:
import pandas as pd
import numpy as np
import random
from tqdm import tqdm
import seaborn as sns
import matplotlib.pyplot as plt
import dill as pickle

from scipy.stats import pearsonr
from scipy import stats
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor
from sklearn.linear_model import LinearRegression
from sklearn import preprocessing
from sklearn.tree import DecisionTreeRegressor

In [3]:
class options:
    def __init__(self):
#         self.datapath = './third_party/BPPPG/f_features_1234.csv'
        self.dataset_name = 'vitaldb'
        self.filter_name = 'zhang21'
        self.alg_name = 'hasandazeh19'
        self.datapath = '../results/features/features_' + self.dataset_name + '_' + self.filter_name + '_' + self.alg_name + '.csv'
        
        if ('provided' in self.dataset_name) or ('features' in self.dataset_name):
            self.sbp_col = '4'
            self.dbp_col = '5'
            self.feature_cols = np.arange(11, 101).astype(str)
        else:
            self.sbp_col = 'SBP'
            self.dbp_col = 'DBP'

            df = pd.read_csv(self.datapath)
            self.feature_cols = df.columns[(df.columns != 'SBP') & (df.columns != 'DBP') & 
                                           (df.columns != 'MBP') & (df.columns != 'segment_num') 
                                           & (df.columns != 'subject_id')
                                          ]

        self.k_folds = 10
        self.scaler = preprocessing.StandardScaler()
        self.sbp_models = [RandomForestRegressor(n_jobs=4,n_estimators=100), 
#                            AdaBoostRegressor(n_estimators=200),
#                            LinearRegression(n_jobs=4),
#                            DecisionTreeRegressor()
                          ]
        self.sbp_model_names = ['RandomForest', 
#                                 'AdaBoost', 'Linear', 'DecisionTree'
                               ]
        self.dbp_models = [RandomForestRegressor(n_jobs=4,n_estimators=100), 
                           AdaBoostRegressor(n_estimators=200),
                           LinearRegression(n_jobs=4),
                           DecisionTreeRegressor()
                          ]
        self.dbp_model_names = ['RandomForest', 
#                                 'AdaBoost', 'Linear', 'DecisionTree'
                               ]

In [4]:
opt = options()
df = pd.read_csv(opt.datapath)

# make changes from provided features.csv on github
# available at https://github.com/navidhasanzadeh/BPPPG
# features extracted from script have been filtered, so this part is unneeded
if ('provided' in opt.datapath) or ('features' in opt.dataset_name):
    df = df.drop(['0','1','2','3','6','7','8','9'], axis=1)
    df = df.replace([np.inf, -np.inf],np.nan)
    df = df.fillna(df.mean())
    df = df.reset_index(drop=True)
    df = df.drop(df[df['4'] < 90].index)
    df = df.drop(df[df['5'] < 40].index)
    df = df.drop(df[df['10'] < 50].index)
    df = df.drop(df[df['4'] > 200].index)
    df = df.drop(df[df['5'] > 100].index)
    df = df.drop(df[df['10'] > 140].index)
    df = df.drop(['10'], axis=1)
else:
    df = df.drop(df[df[opt.sbp_col] < 90].index)
    df = df.drop(df[df[opt.dbp_col] < 40].index)
    df = df.drop(df[df[opt.sbp_col] > 200].index)
    df = df.drop(df[df[opt.dbp_col] > 100].index)
feature_df = df.copy()


fold_size = int(len(feature_df)/opt.k_folds)
sbp_std = feature_df[opt.sbp_col].std()
dbp_std = feature_df[opt.dbp_col].std()
sbp_model_results = {}
dbp_model_results = {}
for m in tqdm(range(len(opt.sbp_models))):
    sbp_ests = np.array([])
    sbp_gts = np.array([])
    sbp_maes = np.array([])
    sbp_mes = np.array([])
    sbp_raw_model_result = {}
    dbp_ests = np.array([])
    dbp_gts = np.array([])
    dbp_maes = np.array([])
    dbp_mes = np.array([])
    dbp_raw_model_result = {}
    print('Training Models: ' + opt.sbp_model_names[m] + ' for SBP and ' + opt.sbp_model_names[m] + ' for DBP')
    for i in tqdm(range(1, opt.k_folds)):
        print('Training ' + 'fold number ' + str(i))
        trainData = feature_df.loc[0:i*fold_size]
        trainData = trainData.append(feature_df.loc[(i+1)*fold_size:])
        testData = feature_df.loc[i*fold_size:(i+1)*fold_size]
        Xy_cols = np.append([opt.sbp_col, opt.dbp_col], opt.feature_cols)
        trainData = trainData[Xy_cols]
        testData = testData[Xy_cols]

        X_train = np.array(trainData[opt.feature_cols])
        y_train = np.array(trainData[[opt.sbp_col, opt.dbp_col]])
        X_test = np.array(testData[opt.feature_cols])
        y_test = np.array(testData[[opt.sbp_col, opt.dbp_col]])    

        scaler = opt.scaler.fit(X_train)
        X_train = scaler.transform(X_train) 
        X_test = scaler.transform(X_test)   
        
        X_train[np.isinf(X_train)] = 0
        X_train[np.isnan(X_train)] = 0
        X_test[np.isinf(X_test)] = 0
        X_test[np.isnan(X_test)] = 0

        sbp_model = opt.sbp_models[m]
        sbp_model.fit(X_train, y_train[:, 0])
        sbp_est = sbp_model.predict(X_test) 
        dbp_model = opt.dbp_models[m]
        dbp_model.fit(X_train, y_train[:, 1])
        dbp_est = dbp_model.predict(X_test)
        
        sbp_gts = np.append(sbp_gts, y_test[:, 0])
        sbp_ests = np.append(sbp_ests, sbp_est)
        sbp_mes = np.append(sbp_mes, sbp_est - y_test[:, 0])
        dbp_gts = np.append(dbp_gts, y_test[:, 1])
        dbp_ests = np.append(dbp_ests, dbp_est)
        dbp_mes = np.append(dbp_mes, dbp_est - y_test[:, 1])
    
    sbp_model_result = {}
    sbp_model_result['raw ests'] = sbp_ests
    sbp_model_result['raw gts'] = sbp_gts
    sbp_model_result['bias'] = (sbp_ests - sbp_gts).mean()
    sbp_model_result['err std'] = (sbp_ests - sbp_gts).std()
    sbp_model_result['ED'] = sbp_std/(sbp_ests - sbp_gts).std()
    sbp_model_result['dist std'] = sbp_std
    sbp_model_results[opt.sbp_model_names[m]] = sbp_model_result
    dbp_model_result = {}
    dbp_model_result['raw ests'] = dbp_ests
    dbp_model_result['raw gts'] = dbp_gts
    dbp_model_result['bias'] = (dbp_ests - dbp_gts).mean()
    dbp_model_result['err std'] = (dbp_ests - dbp_gts).std()
    dbp_model_result['ED'] = dbp_std/(dbp_ests - dbp_gts).std()
    sbp_model_result['dist std'] = sbp_std
    dbp_model_results[opt.dbp_model_names[m]] = dbp_model_result

  0%|          | 0/9 [00:00<?, ?it/s]/opt/conda/lib/python3.8/site-packages/sklearn/utils/extmath.py:1000: RuntimeWarning: overflow encountered in square
  temp **= 2
/opt/conda/lib/python3.8/site-packages/sklearn/utils/extmath.py:1006: RuntimeWarning: overflow encountered in square
  new_unnormalized_variance -= correction**2 / new_sample_count
/opt/conda/lib/python3.8/site-packages/sklearn/utils/extmath.py:1006: RuntimeWarning: invalid value encountered in subtract
  new_unnormalized_variance -= correction**2 / new_sample_count
/opt/conda/lib/python3.8/site-packages/sklearn/preprocessing/_data.py:84: RuntimeWarning: overflow encountered in square
  upper_bound = n_samples * eps * var + (n_samples * mean * eps) ** 2


Training Models: RandomForest for SBP and RandomForest for DBP
Training fold number 1



 11%|█         | 1/9 [00:09<01:15,  9.42s/it]/opt/conda/lib/python3.8/site-packages/sklearn/utils/extmath.py:1000: RuntimeWarning: overflow encountered in square
  temp **= 2
/opt/conda/lib/python3.8/site-packages/sklearn/utils/extmath.py:1006: RuntimeWarning: overflow encountered in square
  new_unnormalized_variance -= correction**2 / new_sample_count
/opt/conda/lib/python3.8/site-packages/sklearn/utils/extmath.py:1006: RuntimeWarning: invalid value encountered in subtract
  new_unnormalized_variance -= correction**2 / new_sample_count
/opt/conda/lib/python3.8/site-packages/sklearn/preprocessing/_data.py:84: RuntimeWarning: overflow encountered in square
  upper_bound = n_samples * eps * var + (n_samples * mean * eps) ** 2


Training fold number 2



 22%|██▏       | 2/9 [00:18<01:05,  9.38s/it]/opt/conda/lib/python3.8/site-packages/sklearn/utils/extmath.py:1000: RuntimeWarning: overflow encountered in square
  temp **= 2
/opt/conda/lib/python3.8/site-packages/sklearn/utils/extmath.py:1006: RuntimeWarning: overflow encountered in square
  new_unnormalized_variance -= correction**2 / new_sample_count
/opt/conda/lib/python3.8/site-packages/sklearn/utils/extmath.py:1006: RuntimeWarning: invalid value encountered in subtract
  new_unnormalized_variance -= correction**2 / new_sample_count
/opt/conda/lib/python3.8/site-packages/sklearn/preprocessing/_data.py:84: RuntimeWarning: overflow encountered in square
  upper_bound = n_samples * eps * var + (n_samples * mean * eps) ** 2


...


 78%|███████▊  | 7/9 [01:05<00:18,  9.41s/it]/opt/conda/lib/python3.8/site-packages/sklearn/utils/extmath.py:1000: RuntimeWarning: overflow encountered in square
  temp **= 2
/opt/conda/lib/python3.8/site-packages/sklearn/utils/extmath.py:1006: RuntimeWarning: overflow encountered in square
  new_unnormalized_variance -= correction**2 / new_sample_count
/opt/conda/lib/python3.8/site-packages/sklearn/utils/extmath.py:1006: RuntimeWarning: invalid value encountered in subtract
  new_unnormalized_variance -= correction**2 / new_sample_count
/opt/conda/lib/python3.8/site-packages/sklearn/preprocessing/_data.py:84: RuntimeWarning: overflow encountered in square
  upper_bound = n_samples * eps * var + (n_samples * mean * eps) ** 2


Training fold number 8



 89%|████████▉ | 8/9 [01:15<00:09,  9.41s/it]/opt/conda/lib/python3.8/site-packages/sklearn/utils/extmath.py:1000: RuntimeWarning: overflow encountered in square
  temp **= 2
/opt/conda/lib/python3.8/site-packages/sklearn/utils/extmath.py:1006: RuntimeWarning: overflow encountered in square
  new_unnormalized_variance -= correction**2 / new_sample_count
/opt/conda/lib/python3.8/site-packages/sklearn/utils/extmath.py:1006: RuntimeWarning: invalid value encountered in subtract
  new_unnormalized_variance -= correction**2 / new_sample_count
/opt/conda/lib/python3.8/site-packages/sklearn/preprocessing/_data.py:84: RuntimeWarning: overflow encountered in square
  upper_bound = n_samples * eps * var + (n_samples * mean * eps) ** 2


Training fold number 9



100%|██████████| 1/1 [01:24<00:00, 84.84s/it]


In [5]:
pickle_dict = {'sbp': sbp_model_results, 'dbp': dbp_model_results, 'opt': opt}
with open('../results/training/' + opt.dataset_name + '_' + opt.filter_name + '_' + opt.alg_name + '.pickle', 'wb') as f:
    pickle.dump(pickle_dict, f, protocol=pickle.HIGHEST_PROTOCOL)